In [ ]:
import csv
import ast
import numpy as np
import time

start = time.time()

# --- Load BB (Billboard data) ---
BB = []
with open("BB_LA.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:   # ✅ Loop starts here
        BB.append({
            'idx': int(row['B_index']),
            'cost': float(row['Cost']),
            'Influence': float(row['Influence'])   # ✅ Now this is valid
        })


# Sort billboard data by descending influence
BB.sort(key=lambda x: x['Influence'], reverse=True)
# print(BB)

# --- Load ub (User influence data) ---
ub = []
with open("new_ub.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row['Influenced Billboards'] = ast.literal_eval(row['Influenced Billboards'])
        ub.append(row)

# --- Load Demand and Budget (from advertiser_1.csv) ---
Dem = []
Budget = []
with open("Advertiser_LA1.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        Dem.append(float(row['Demand']))
        Budget.append(float(row['Payment']))

print("Budgets:", Budget)

# BillboardSlot class
class BillboardSlot:
    def __init__(self, idx):
        self.idx = idx
    def __repr__(self):
        return f"Slot({self.idx})"

# Convert BB to BillboardSlot objects using B_index
BS = [BillboardSlot(item['idx']) for item in BB]
# print("BS",BS)

Budgets: [80.0, 120.0, 160.0, 80.0, 80.0, 160.0, 160.0, 80.0, 80.0, 120.0, 120.0, 120.0, 40.0, 80.0, 120.0, 160.0, 80.0, 80.0, 160.0, 120.0, 120.0, 80.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 40.0, 120.0, 80.0, 80.0, 80.0, 120.0, 120.0, 80.0, 160.0, 80.0, 120.0, 80.0, 120.0, 80.0, 40.0, 120.0, 120.0, 120.0, 80.0, 160.0, 80.0, 160.0, 80.0, 160.0, 120.0, 80.0, 80.0, 160.0, 80.0, 120.0, 80.0, 120.0, 120.0, 80.0, 120.0, 80.0, 80.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 120.0, 160.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 160.0, 160.0, 80.0, 160.0, 120.0, 80.0, 80.0, 120.0]


In [2]:
# Influence function
def influence(S):
    influence_score = 0.0
    slot_ids = {s.idx if isinstance(s, BillboardSlot) else s for s in S}
    # print("slot_ids",slot_ids)
    for row in ub:
        influenced_billboards = row['Influenced Billboards']
        intersection_result = {int(key): value for key, value in influenced_billboards.items() if int(key) in slot_ids}
        product = 1.0
        # print("Intersection Result:",intersection_result)
        for value in intersection_result.values():
            # print("vlaue",value)
            
            product *= (1 - value)
            # print("product",product)
        influence_score += (1 - product)
        # print("influence_score",influence_score)
    return influence_score

In [3]:
def assign_top_influential(BS, Dem, Budget):
    # Mapping slot.idx → cost
    slot_cost_map = {slot['idx']: slot['cost'] for slot in BB}

    used_slots = set()  # Globally used slot indices
    Si_list = [set() for _ in Dem]  # Slots assigned per product
    
    index = 0  # Index into sorted BS list

    for j in range(len(Dem)):
        demand = Dem[j]
        current_influence = 0.0
        remaining_budget = Budget[j]  # Track remaining budget for each product
        
        while current_influence < demand and index < len(BS):
            slot = BS[index]
            index += 1

            # Skip if slot is already used
            if slot.idx in used_slots:
                continue

            cost = slot_cost_map.get(slot.idx, 0.0)

            
            if cost > remaining_budget:
                continue  # Skip if too costly

            # Assign slot
            Si_list[j].add(slot.idx)
            used_slots.add(slot.idx)
            remaining_budget -= cost  # Deduct cost from remaining budget

            # Update influence
            current_influence = influence(Si_list[j])

        # Optional warning
        if current_influence < demand:
            print(f"[!] Product {j}: Demand not fully satisfied → Influence = {current_influence:.2f}, Demand = {demand:.2f}, Budget = {Budget[j]:.2f}")

    return Si_list


In [4]:
# Run the budget-constrained top-k allocation
result = assign_top_influential(BS, Dem, Budget)
total_influence = 0.0

# Print per-product results with Demand
for i, slots in enumerate(result):
    slot_cost = sum(BB[slot]['cost'] for slot in slots if slot < len(BB))
    achieved_influence = influence(slots)
    total_influence += achieved_influence
    print(f"Product {i+1}: Demand = {Dem[i]:.2f}, Influence = {achieved_influence:.2f}, "
          f"Cost = {slot_cost:.2f}, Budget = {Budget[i]:.2f}, Slots = {sorted(list(slots))}")

# Total unique slots and influence
all_selected = set().union(*result)
print("\nTotal Unique Slots Selected:", len(all_selected))
print("Total Combined Influence:", total_influence)
print("Execution Time:", round(time.time() - start, 2), "seconds")

# Count satisfied products
satisfied_count = sum(1 for j in range(len(Dem)) if influence(result[j]) >= Dem[j])
print("Number of products whose demands are satisfied:", satisfied_count, "out of", len(Dem))

Product 1: Demand = 3.00, Influence = 28.54, Cost = 0.00, Budget = 80.00, Slots = [9242]
Product 2: Demand = 4.00, Influence = 18.00, Cost = 0.01, Budget = 120.00, Slots = [1843]
Product 3: Demand = 4.00, Influence = 18.00, Cost = 0.01, Budget = 160.00, Slots = [1844]
Product 4: Demand = 3.00, Influence = 10.00, Cost = 0.01, Budget = 80.00, Slots = [1775]
Product 5: Demand = 3.00, Influence = 9.35, Cost = 0.00, Budget = 80.00, Slots = [9253]
Product 6: Demand = 4.00, Influence = 16.00, Cost = 0.01, Budget = 160.00, Slots = [1772]
Product 7: Demand = 4.00, Influence = 16.00, Cost = 0.01, Budget = 160.00, Slots = [1822]
Product 8: Demand = 3.00, Influence = 8.00, Cost = 0.01, Budget = 80.00, Slots = [1841]
Product 9: Demand = 3.00, Influence = 8.00, Cost = 0.01, Budget = 80.00, Slots = [1851]
Product 10: Demand = 4.00, Influence = 16.00, Cost = 0.01, Budget = 120.00, Slots = [1861]
Product 11: Demand = 4.00, Influence = 16.00, Cost = 0.01, Budget = 120.00, Slots = [1885]
Product 12: Dema

In [5]:
h = len(Dem)
count = 0

for j in range(h):
    if influence(result[j]) >= Dem[j]:
        count += 1

print("Number of products whose demands are satisfied:", count, "out of", h)


Number of products whose demands are satisfied: 100 out of 100
